### Protocolos Clínicos de ARIA
# Procesamiento de prescripciones
------
**Objeto**: Extraer los datos de las prescripciones de pacientes para generar protocolos clínicos

Importación de módulos

In [1]:
import pandas as pd
import io
import re

Cambiar el directorio de trabajo, asumimos que el cuaderno está en el directorio `tools` dentro de la carpeta del proyecto

In [2]:
%cd ..

/Users/cesar/Documents/Development/rtpros


/Users/cesar/Documents/Development/rtps/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Leer los datos de una prescripción exportada de ARIA en un DataFrame de Pandas

In [4]:
prdf = pd.read_csv('prescripciones/orl_radical.csv')

## Órganos de riesgo
Los campos relativos a los órganos de riesgo están contenidos en el campo `OrganAtRisk`

Los separamos por líneas. La información contenida en cada línea no sigue un patrón regular

In [5]:
oar_lines = prdf.OrgansAtRisk.values[0].split('\n')

Definir un diccionario de expresión regulares para identificar la línea que encabeza la información de un órgano concreto y filtrar el nombre del órgano, la dosis media y la dosis máxima

In [6]:
rx_dict = {
    'Organ': re.compile(r'Organ :(?P<Organ>.*) Mean'),
    'Dmean': re.compile(r'Mean :(?P<Dmean>.*) Max Dose'),
    'Dmax' : re.compile(r'Max Dose :(?P<Dmax>.*)$'),
}

Definir una función que identifica el encabezado de cada órgano

In [7]:
def _parse_line(line):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """

    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

Definir una función que procesando la línea de encabezado de un órgano genera un diccionario con el nombre del órgano, y los objetivos de dosis media y dosis máxima

In [8]:
def _parse_organ(line):
    """
    Do a regex search against all defined regexes and
    return a dictionary the key and match result

    """

    matches = {}
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            matches[key] = match.group(key)
        
    return matches

Procesar los órganos de riesgo generando una lista de sublistas.

Cada sublista commienza con la línea de encabezado de cada órgano. A continuación contiene una línea irrelevante con la indicación 'Constrains:' y depués una línea con cada restricción dosimétrica

In [9]:
oars, oar = [], None
for oar_line in oar_lines:
    oar_key, oar_name = _parse_line(oar_line)
    if oar_key:
        oars.append(oar)
        oar = [oar_line]
    else:
        oar.append(oar_line)
oars.remove(None)
oars

[['Organ :parótida derecha Mean : 26 Gy Max Dose : ',
  'Constraints : ',
  'V30$50 %'],
 ['Organ :parótida izda Mean : 26 Gy Max Dose : ',
  'Constraints : ',
  'V30$50 %'],
 ['Organ :médula Mean :  Max Dose : 45 Gy', 'Constraints : ', ''],
 ['Organ :tronco cerebral Mean :  Max Dose : ',
  'Constraints : ',
  'D53$100%',
  'V60$30%'],
 ['Organ :cristalino derecho Mean :  Max Dose : 8 Gy', 'Constraints : ', ''],
 ['Organ :cristalino izquierdo Mean :  Max Dose : 8 Gy', 'Constraints : ', ''],
 ['Organ :oido interno derecho Mean :  Max Dose : 50 Gy',
  'Constraints : ',
  ''],
 ['Organ :oido interno izquierdo Mean :  Max Dose : 50 Gy',
  'Constraints : ',
  ''],
 ['Organ :laringe Mean :  Max Dose : 70 Gy', 'Constraints : ', ''],
 ['Organ :mandíbula Mean :  Max Dose : 70 Gy', 'Constraints : ', ''],
 ['Organ :tiroides Mean :  Max Dose : 45 Gy', 'Constraints : ', ''],
 ['Organ :quiasma óptico Mean :  Max Dose : 50 Gy', 'Constraints : ', ''],
 ['Organ :n óptico derecho Mean :  Max Dose : 50 G

In [10]:
oars_list = []
for oar in oars:
    oar_dict = _parse_organ(oar[0])
    oar_dict['DosimPars'] = oar[2:]
    oars_list.append(oar_dict)
oardf = pd.DataFrame(oars_list)
oardf

,Organ,Dmean,Dmax,DosimPars
0,parótida derecha,26 Gy,,[V30$50 %]
1,parótida izda,26 Gy,,[V30$50 %]
2,médula,,45 Gy,[]
3,tronco cerebral,,,"[D53$100%, V60$30%]"
4,cristalino derecho,,8 Gy,[]
5,cristalino izquierdo,,8 Gy,[]
6,oido interno derecho,,50 Gy,[]
7,oido interno izquierdo,,50 Gy,[]
8,laringe,,70 Gy,[]
9,mandíbula,,70 Gy,[]


In [11]:
for dpar in oardf[oardf.Organ == 'tronco cerebral'].DosimPars.values[0]:
    print(dpar)

D53$100%
V60$30%


## Especificaciones de cobertura
Los campos relativos a las especificaciens de cobertura están contenidos en el campo `CoverageConstriants`

Toda la información se guarda en una única cadena. Cada campo está separado por '|'.

Separamos los campos por líneas. La información contenida en cada línea sigue un patrón regular:
* Nombre del volumen tras el texto *Volume / Structure :*
* Valor de la dosis mínima en Gy tras el texto *Min Dose:*
* Valor de la dosis máxima en Gy tras el texto *Max Dose:*
* Restricción de cobertura mínima tras el texto *At Least* en porcentaje de volumen para un porcentaje de dosis y una dosis absoluta
* Restricción sobre los máximos de dosis tras el texto *NoMore* en porcentaje de volumen para un porcentaje de dosis y una dosis absoluta

In [20]:
cc_lines = prdf.CoverageConstraints.values[0].split('|')

In [21]:
for cc_line in cc_lines:
    print(cc_line)

 Volume / Structure :PTV70 Min Dose:   Gy Max Dose:   Gy At Least 95 % of PTV70 at 95 % 55.195 Gy No More Than 5 % of PTV70 at 107 % 62.167 Gy
 Volume / Structure :PTV58.1 Min Dose:   Gy Max Dose:   Gy At Least 95 % of PTV58.1 at 95 % 66.5 Gy No More Than 5 % of PTV58.1 at 107 % 74.9 Gy


Definir un diccionario de expresiones regulares para analizar cada línea de especificación de restricciones y filtrar el nombre del volumen, la dosis mínima , la dosis máxima, la cobertura mínima y la restricción sobre el máximo volumen permitido con una dosis dada.

In [41]:
cc_rx_dict = {
    'Volume': re.compile(r'Volume / Structure :(?P<Volume>.*) Min Dose'),
    'Min': re.compile(r'Min Dose:(?P<Min>.*) Gy Max'),
    'Max': re.compile(r'Max Dose:(?P<Max>.*) Gy At'),
    'AtLeast': re.compile(r'At Least (?P<AtLeast>.*) % of (?P<Volume>.*) at (?P<Percentage>.*) % (?P<Dose>.*) Gy No More Than'),
    'NoMore': re.compile(r'No More Than (?P<NoMore>.*) % of (?P<Volume>.*) at (?P<Percentage>.*) % (?P<Dose>.*) Gy'),
}

Definir una función para procesar las líneas de especificaciones de restricciones

In [45]:
def _parse_volume(line):
    """
    Do a regex search against all defined regexes and
    return a dictionary the key and match result

    """

    matches = {}
    for key, rx in cc_rx_dict.items():
        match = rx.search(line)
        if match:
            if key == 'Volume':
                volume = match.group(key)
                matches[key] = match.group(key)
            elif key == 'AtLeast' and match.group('Volume') == volume:
                constraint = [match.group('AtLeast'), match.group('Percentage'), match.group('Dose')]
                matches[key] = constraint
            elif key == 'NoMore' and match.group('Volume') == volume:
                constraint = [match.group('NoMore'), match.group('Percentage'), match.group('Dose')]
                matches[key] = constraint
            else:
                matches[key] = match.group(key)
        
    return matches

In [49]:
pd.DataFrame([_parse_volume(cc_line) for cc_line in cc_lines])

,Volume,Min,Max,AtLeast,NoMore
0,PTV70,,,"[95, 95, 55.195]","[5, 107, 62.167]"
1,PTV58.1,,,"[95, 95, 66.5]","[5, 107, 74.9]"
